<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/5-Fine%20Tuning/5_5_Optimizing_Prompt_Tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large Language Models Projects
### Apply and Implement Strategies for Large Language Models
## 5.5-Optimizing Prompt Tokens.

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)

## Loading the Peft Library
This library contains the Hugging Face implementation of various fine-tuning techniques, including Prompt Tuning

In [1]:
!pip install -q peft==0.11.1
!pip install -q datasets==2.20.0
!pip install -q accelerate==0.32.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 968.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 whic

From the transformers library, we import the necessary classes to instantiate the model and the tokenizer.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

## Loading the model and the tokenizers.

Bloom is one of the smallest and smartest models available for training with the PEFT Library using Prompt Tuning.

I'm opting for the smallest one to minimize training time and avoid memory issues in Colab. Feel Free to try with a bigger one if you have acces to a good GPU.

In [101]:
model_name = "bigscience/bloomz-560m"
#model_name = "bigscience/bloomz-1b1"

NUM_VIRTUAL_TOKENS = 20
#If you just want to test the solution, you can reduce the EPOCHs.
NUM_EPOCHS_CLASSIFIER = 10
device = "cuda" #Replace with "mps" for Silicon chips.

In [102]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map = device
)

## Inference with the pre trained bloom model



In [103]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        #repetition_penalty=1.1, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

To compare the pre-trained model with the same model after the prompt-tuning process, I will run the same sentence on both models.

The model doesn't know what its mission is and answers as best as it can. It's not a bad response, but it's not what we're looking for.

# Hate Classifier


In [105]:
prompt = """
You are a highly accurate and efficient moderator.
Your task is to detect hate speech in a given sentence.
Respond with "hate detected" or "no hate detected" as appropriate.

Sentence: {sentence} Label:
"""

In [106]:
sentence = "I Dont Like short people, I have no idea why they exist."
input_classifier = tokenizer(prompt.format(sentence=sentence), return_tensors="pt")

In [107]:
#input_classifier = tokenizer("Sentence : Head is the shape of a light bulb. Label : ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device))

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

['\nYou are a highly accurate and efficient moderator. \nYour task is to detect hate speech in a given sentence. \nRespond with "hate detected" or "no hate detected" as appropriate.\n\nSentence: I Dont Like short people, I have no idea why they exist. Label:\nNo hate detected']


The model has no idea what its purpose is, so it completes the sentence as best as it can.

##Loading the Dataset

* https://huggingface.co/datasets/SetFit/ethos_binary

In [110]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [112]:
from datasets import load_dataset
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(
    data_classifier['train'])

data_classifier = data_classifier.map(
    lambda samples: tokenizer(samples["text"]),
    batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(
    ['label', 'label_text', 'text'])

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/598 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [113]:
data_classifier

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 598
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [114]:
train_sample_classifier

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 598
})

I have deleted all the columns from the dataset that are not strictly necessary for training, that is to say, I have removed all columns that are not essential for the model's learning process.

In [115]:
print(train_sample_classifier[2:3])

{'input_ids': [[62121, 1671, 915, 44478, 632, 368, 32428, 461, 267, 12490, 159845, 77658, 915, 654, 74549, 40423]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


## prompt-tuning configuration

In [ ]:
generation_config_classifier = PromptTuningConfig(
    #This type indicates the model will generate text.
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Indicate if the text contains hate speech or no hate speech.",
    #Number of virtual tokens to be added and trained.
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    #The pre-trained model.
    tokenizer_name_or_path=model_name
)

In [ ]:
peft_model_classifier = get_peft_model(
    foundational_model,
    generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())

trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144
None


In [ ]:
if not os.path.exists(output_directory_classifier):
    os.mkdir(output_directory_classifier)

In [ ]:
training_args_classifier = create_training_arguments(
    output_directory_classifier,
    3e-2,
    NUM_EPOCHS_CLASSIFIER)

## Training Second Model

In [ ]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss
500,3.299800
1000,3.263300
1500,3.233000
2000,3.190600
2500,3.184800


TrainOutput(global_step=2990, training_loss=3.2188521063048703, metrics={'train_runtime': 235.485, 'train_samples_per_second': 25.394, 'train_steps_per_second': 12.697, 'total_flos': 667918083244032.0, 'train_loss': 3.2188521063048703, 'epoch': 10.0})

In [ ]:
trainer_classifier.model.save_pretrained(output_directory_classifier)

## Inference second Model

In [ ]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [ ]:
loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier,
                                             max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))

['Sentence : Head is the shape of a light bulb. Label :  no hate speech']


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Let's check how the model's response has changed with training:

**Input for the model**
```
Sentence : Head is the shape of a light bulb. Label :
Sentence : I don't like short people, no idea why they exist. Label :
```

**Original model**
```
Sentence : Head is the shape of a light bulb. Label :  head
Sentence : I don't like short people, no idea why they exist. Label :  No
```
**Trained for classification with Prompt-tuning**
```
Sentence : Head is the shape of a light bulb. Label :  no hate speech
Sentence : I don't like short people, no idea why they exist. Label :  hate speech
```

It's clear that the training has fulfilled its purpose. The original model doesn't know what its mission is and tries to complete the sentences as best as it can. On the other hand, the updated model with prompt-tuning does know what its mission is and is able to classify the sentences correctly and in the indicated format.


# Conclusion
Prompt Tuning is an amazing technique that can save us hours of training and a significant amount of money. In the notebook, we have trained two models in just a few minutes, and we can have both models in memory, providing service to different clients.

If you want to try different combinations and models, the notebook is ready to use another model from the Bloom family.

You can change the number of epochs to train, the number of virtual tokens, and the model. However, there are many configurations to change.

*The responses of the fine-tuned models may vary every time we train them. I've pasted the results of one of my trainings, but the actual results may differ.*